# Mask and Social Distancing Detection 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from scipy.spatial import distance
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
from sklearn.model_selection import train_test_split
import os

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("../input"))
print(os.listdir("../working"))


# Any results you write to the current directory are saved as output.

**LOADING THE HAAR CASACADE CLASSIFIER**

In [3]:
#loading haarcascade_frontalface_default.xml
face_model = cv2.CascadeClassifier('../input/haarcascades/haarcascade_frontalface_default.xml')

**TESTING HAAR CASCADE ON A SAMPLE IMAGE**


In [5]:
#trying it out on a sample image
img = cv2.imread('../input/face-mask-detection/images/maksssksksss244.png')

img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)

#returns a list of (x,y,w,h) tuples
faces = face_model.detectMultiScale(img,scaleFactor=1.1, minNeighbors=4) 

out_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #colored output image

#plotting
for (x,y,w,h) in faces:
    cv2.rectangle(out_img,(x,y),(x+w,y+h),(0,0,255),1)

plt.figure(figsize=(12,12))
plt.imshow(out_img)

# For Social Distancing


In [6]:
MIN_DISTANCE = 130

In [7]:
if len(faces)>=2:
    label = [0 for i in range(len(faces))]
    for i in range(len(faces)-1):
        for j in range(i+1, len(faces)):
            dist = distance.euclidean(faces[i][:2],faces[j][:2])
            if dist<MIN_DISTANCE:
                label[i] = 1
                label[j] = 1
    new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #colored output image
    for i in range(len(faces)):
        (x,y,w,h) = faces[i]
        if label[i]==1:
            cv2.rectangle(new_img,(x,y),(x+w,y+h),(255,0,0),1)
        else:
            cv2.rectangle(new_img,(x,y),(x+w,y+h),(0,255,0),1)
    plt.figure(figsize=(10,10))
    plt.imshow(new_img)
            
else:
    print("No. of faces detected is less than 2")

#### Here the Red box shows violation of social distancing.

# **Using VGG19 for mask detection**


In [8]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [9]:
#Load train and test set
train_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
test_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test'
val_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'

In [10]:
# Data augmentation

train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=0.2,shear_range=0.2)
train_generator = train_datagen.flow_from_directory(directory=train_dir,target_size=(128,128),class_mode='categorical',batch_size=32)

val_datagen = ImageDataGenerator(rescale=1.0/255)
val_generator = train_datagen.flow_from_directory(directory=val_dir,target_size=(128,128),class_mode='categorical',batch_size=32)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = train_datagen.flow_from_directory(directory=val_dir,target_size=(128,128),class_mode='categorical',batch_size=32)

### Building VGG19 transfer learning model.

In [11]:
vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(128,128,3))

for layer in vgg19.layers:
    layer.trainable = False
    
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.summary()

In [12]:
from tensorflow import keras;    
from tensorflow.keras import metrics;
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=['acc', keras.metrics.Precision(), keras.metrics.Recall()])

In [13]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=len(train_generator)//32,
                              epochs=20,validation_data=val_generator,
                              validation_steps=len(val_generator)//32)

In [14]:
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_generator, steps=7)

### Testing the model on the test data

In [15]:
img = cv2.imread('../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/1565.png')
plt.imshow(img)
img = cv2.resize(img, (128,128)) 
img = np.reshape(img, [1,128,128,3])/255.0

pred_prob = model.predict(img)

pred_prob

The model is able to classify if the person is wearing a mask or not.

### Save the model.

In [16]:
model.save('masknet.h5')

# **Integrating with haar cascade**

In [17]:
mask_label = {0:'MASK',1:'NO MASK'}
dist_label = {0:(0,255,0),1:(255,0,0)}

In [18]:
def mask_social(img):
    
    faces = face_model.detectMultiScale(img, scaleFactor=1.1, minNeighbors=4)
    
    if len(faces)>=2:
        label = [0 for i in range(len(faces))]
    
        for i in range(len(faces)-1):
            for j in range(i+1, len(faces)):
                dist = distance.euclidean(faces[i][:2],faces[j][:2])
                
                if dist<MIN_DISTANCE:
                    label[i] = 1
                    label[j] = 1
                
        new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #colored output image
    
        for i in range(len(faces)):
            (x,y,w,h) = faces[i]
        
            crop = new_img[y:y+h,x:x+w]
            crop = cv2.resize(crop,(128,128))
            crop = np.reshape(crop,[1,128,128,3])/255.0
        
            mask_result = model.predict(crop)
        
            cv2.putText(new_img,mask_label[mask_result.argmax()],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[label[i]],2)
            cv2.rectangle(new_img,(x,y),(x+w,y+h),dist_label[label[i]],1)
        
        plt.figure(figsize=(10,10))
        plt.imshow(new_img)
            
    else:
        print("No. of faces detected is less than 2 so social distancing detection isn't possible")
    
        new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        for i in range(len(faces)):
            (x,y,w,h) = faces[i]
    
            crop = new_img[y:y+h,x:x+w]
            crop = cv2.resize(crop,(128,128))
            crop = np.reshape(crop,[1,128,128,3])/255.0

            mask_result = model.predict(crop)

            cv2.putText(new_img,mask_label[mask_result.argmax()], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            cv2.rectangle(new_img, (x,y), (x+w,y+h), 1)
    
        plt.figure(figsize=(10,10))
        plt.imshow(new_img)

In [19]:
img1 = cv2.imread('../input/face-mask-detection/images/maksssksksss244.png')
mask_social(img1)

In [20]:
img2 = cv2.imread('../input/face-mask-detection/images/maksssksksss10.png')
mask_social(img2)

# **FACE RECOGNITION**

In [21]:
# extract a single face from a given photograph

def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    
    # convert to RGB, if needed
    image = image.convert('RGB')
    
    # convert to array
    pixels = np.asarray(image)
    
    # resize pixels to the model size
    image = Image.fromarray(pixels)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    
    return face_array

In [22]:
def load_face(dir):
    faces = list()
    paths = list()
    
    # enumerate files
    for filename in os.listdir(dir):
        path = dir + filename
        face = extract_face(path)
        faces.append(face)
        paths.append(path)
    
    return faces, paths

def load_dataset(dir):
    # list for faces and labels
    X, y, paths, t_paths = list(), list(), list(), list()
    
    for subdir in os.listdir(dir):
        path = dir + subdir + '/'
        faces, paths = load_face(path)
        t_paths = t_paths + paths
        labels = [subdir for i in range(len(faces))]
        X.extend(faces)
        y.extend(labels)
        
    return np.asarray(X), np.asarray(y), t_paths

**LOAD AND  TRAIN FOR FACE RECOGNITION**

In [23]:
# load train dataset

trainX, trainy, train_paths = load_dataset('../input/fyprmfrd/RMFRD/AFDB_face_datset/')
print(trainX.shape, trainy.shape)

In [24]:
# load test dataset

testX, testy, test_paths = load_dataset('../input/fyprmfrd/RMFRD/AFDB_masked_face_dataset/')
print(testX.shape, testy.shape, len(test_paths))

**Compress and Save Train and Test Dataset**
 After face extraction process, train and dataset occupied memory and there was a memory error. The reason of this process is to deal with this issue .

In [25]:
# save and compress the dataset for further use
np.savez_compressed('new_modified_masked_face.npz', trainX, trainy, testX, testy)
print('Done with Compression')
del trainX, trainy, testX, testy

In [29]:
data = np.load('/kaggle/working/new_modified_masked_face.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
del data

In [30]:
trainx, valid = train_test_split(trainX, test_size=0.4, random_state=42, shuffle=True)
del trainX
print('Split Training Data')

In [31]:
print("number of image in train dataset : %s" %(len(trainx)))

print("number of image in train dataset : %s" %(len(valid)))

In [32]:
y_train, y_valid = train_test_split(trainy, test_size=0.4, random_state=42, shuffle=True)
del trainy
print('Split Training Data')

In [33]:
print("number of image in train dataset : %s" %(len(y_train)))

print("number of image in train dataset : %s" %(len(y_valid)))

In [34]:
# save and compress the dataset for further use
np.savez_compressed('modified_extracted_masked_unmasked.npz', trainx, y_train, valid, y_valid,testX, testy)
del trainx, y_train, valid, y_valid, testX, testy
print('Done Compressing')

In [36]:
data = np.load('/kaggle/working/modified_extracted_masked_unmasked.npz')
print(data.files)
trainx, y_train, valid, y_valid,testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)
del data

**FACE EMBEDDINGS WITH FACENET**


In [38]:
facenet_model = load_model('/kaggle/input/facenet/keras-facenet/model/facenet_keras.h5')
print('Loaded Model')

In [39]:
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)

In [40]:
def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    
    # make prediction to get embedding
    yhat = model.predict(sample)
    
    return yhat[0]

**CONVERT EACH FACE IN THE TRAINSET INTO EMBEDDING**


In [41]:
emdTrainX = list()

for face in trainx:
    emd = get_embedding(facenet_model, face)
    emdTrainX.append(emd)
    
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)

embValid = list()

for face in valid:
    emd = get_embedding(facenet_model,face)
    embValid.append(emd)
    
embValid = np.asarray(embValid)
print(embValid.shape)

In [42]:
emdTestX = list()

for face in testX:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
    
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)

**COMPRESSS AND SAVE TRAIN AND TEST EMBEDDINGS**


In [43]:
# save arrays to one file in compressed format
np.savez_compressed('embeddings_masked.npz', emdTrainX, y_train, embValid, y_valid, emdTestX, testy)
del emdTrainX, y_train, embValid, y_valid, emdTestX, testy
print('Here')

In [44]:
data = np.load('/kaggle/working/embeddings_masked.npz')
print(data.files)
emdTrainX, y_train, embValid, y_valid, emdTestX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']

In [45]:
print('Loaded: ', emdTrainX.shape, y_train.shape, embValid.shape, y_valid.shape, emdTestX.shape, testy.shape)

**LABELLED ENCODING**


In [46]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle

print("Dataset: train=%d,validation = %d, test=%d" % (emdTrainX.shape[0],embValid.shape[0] ,emdTestX.shape[0]))

# normalize input vectors
in_encoder = Normalizer(norm='l2')
emdTrainX_norm = in_encoder.transform(emdTrainX)
embValid_norm = in_encoder.transform(embValid)
emdTestX_norm = in_encoder.transform(emdTestX)

# label encode targets
out_encoder = LabelEncoder()
encoder_arr = np.append (y_train, 'wangnan')
out_encoder.fit(encoder_arr)

**ENCODING TRAINY AND TESTY WITH FITTED ENCODER**



In [47]:
trainy_enc = out_encoder.transform(y_train)
y_valid_enc = out_encoder.transform(y_valid)
testy_enc = out_encoder.transform(testy)

**Face Classification with SVC**

In [48]:
model = SVC(kernel='linear', probability=True)
model.fit(emdTrainX_norm, trainy_enc)

In [49]:
# predict
yhat_valid = model.predict(embValid_norm)
yhat_test = model.predict(emdTestX_norm)

# score
score_valid = accuracy_score(y_valid_enc, yhat_valid)
score_test = accuracy_score(testy_enc, yhat_test)

# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_valid*100, score_test*100))

In [50]:
#Save the model
filename = 'linear.sav'
pickle.dump(model, open(filename, 'wb'))

In [51]:
loaded_model = pickle.load(open('linear.sav', 'rb'))

**TESTING FACE RECOGNITION**

In [52]:
from random import choice

for i in range(50):
    
    # select a random face from test set
    selection = choice([i for i in range(testX.shape[0])]) 
    random_face = testX[selection]
    random_face_emd = emdTestX_norm[selection]
    random_face_class = testy_enc[selection]
    random_face_name = out_encoder.inverse_transform([random_face_class])
    
    # prediction for the face
    samples = np.expand_dims(random_face_emd, axis=0)
    yhat_class = loaded_model.predict(samples)
    yhat_prob = loaded_model.predict_proba(samples)
    class_index = yhat_class[0]
    
    if class_index <= 228:
        # get name
        class_probability = yhat_prob[0,class_index] * 100
        predict_names = out_encoder.inverse_transform(yhat_class)
        
        if random_face_name[0] == predict_names[0]:
            print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
            print('Expected: %s' % random_face_name[0])
            
            # plot face
            plt.imshow(random_face)
            title = '%s (%.3f)' % (predict_names[0], class_probability)
            plt.title(title)
            plt.show()

#  **INTEGRATION OF MODULES**

In [65]:
def FaceVIDMask_social(random_face, selection, path):
    
    random_face_emd = emdTestX_norm[selection]
    random_face_class = testy_enc[selection]
    random_face_name = out_encoder.inverse_transform([random_face_class])
    
    # prediction for the face
    samples = np.expand_dims(random_face_emd, axis=0)
    yhat_class = loaded_model.predict(samples)
    yhat_prob = loaded_model.predict_proba(samples)
    class_index = yhat_class[0]
    
    class_probability = yhat_prob[0,class_index] * 100
    predict_names = out_encoder.inverse_transform(yhat_class)
        
    if random_face_name[0] == predict_names[0]:
        print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
        print('Expected: %s' % random_face_name[0])
    
        faces = face_model.detectMultiScale(random_face, scaleFactor=1.1, minNeighbors=1)
    
        if len(faces)>=2:
            label = [0 for i in range(len(faces))]
    
            for i in range(len(faces)-1):
                for j in range(i+1, len(faces)):
                    dist = distance.euclidean(faces[i][:2],faces[j][:2])
                
                    if dist<MIN_DISTANCE:
                        label[i] = 1
                        label[j] = 1
                
            new_img = cv2.cvtColor(random_face, cv2.COLOR_RGB2BGR) #colored output image
    
            for i in range(len(faces)):
                (x,y,w,h) = faces[i]
        
                crop = new_img[y:y+h,x:x+w]
                crop = cv2.resize(crop,(128,128))
                crop = np.reshape(crop,[1,128,128,3])/255.0
        
                mask_result = model.predict(crop)
        
                cv2.putText(new_img,mask_label[mask_result.argmax()],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,dist_label[label[i]],2)
                cv2.rectangle(new_img,(x,y),(x+w,y+h),dist_label[label[i]],1)
        
            plt.figure(figsize=(10,10))
            plt.imshow(new_img)
            
        else:
            print("No. of faces detected is less than 2 so social distancing detection isn't possible")
    
            new_img = cv2.cvtColor(random_face, cv2.COLOR_RGB2BGR)
        
            #if len(faces) == 0:
                #image = cv2.imread(path)
                #img = np.resize(image, (128, 128, 3))
                #plt.imshow(image)
                #img = np.array(img)
                #img = np.reshape(img,[-1,128,128,3])
                #img = img / 255.0

                #pred_prob = model.predict(img)[0]
                #pred_class = list(pred_prob).index(max(pred_prob)) 
                
                #if pred_class == 0:
                    #mask_class = "Wearing Mask"
                #else:
                    #mask_class = "Not Wearing Mask"
    
                #print(mask_class)
        
            #else:
            for i in range(len(faces)):
                (x,y,w,h) = faces[i]
    
                crop = new_img[y:y+h,x:x+w]
                crop = cv2.resize(crop,(128,128))
                crop = np.reshape(crop,[1,128,128,3])/255.0

                mask_result = model.predict(crop)

                cv2.putText(new_img,mask_label[mask_result.argmax()], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                cv2.rectangle(new_img, (x,y), (x+w,y+h), 1)
    
        plt.figure(figsize=(10,10))
        plt.imshow(new_img)

In [74]:
for i in range(10):
    
    # select a random face from test set
    selection = choice([i for i in range(testX.shape[0])]) 
    random_face = testX[selection]
    path = test_paths[selection]
    
    FaceVIDMask_social(random_face, selection, path)